In [10]:
import torch

print(torch.__version__)

2.1.1+cpu


In [1]:
import llm

app = llm.LargeLanguageModel('./../../../data')
history = []
resp, conversation = app.create_conversation("what is a golden", history)
print(resp, conversation)


d:\Git\AIML_Capstone_Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing...
Initializing BitsAndBytesConfig
Initializing model anakin87/zephyr-7b-alpha-sharded


Loading checkpoint shards: 100%|██████████| 8/8 [00:07<00:00,  1.10it/s]


Initializing tokenizer
Reading PDFs


ValueError: Expected IDs to be a non-empty list, got []

In [1]:

import os

import chromadb
import torch
from langchain import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFDirectoryLoader, TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, pipeline)

d:\Git\AIML_Capstone_Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "anakin87/zephyr-7b-alpha-sharded"

def load_model():
        
    print('Initializing BitsAndBytesConfig')

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=False,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float32  # changed from torch.bfloat16
    )

    print('Initializing model', model_name)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=False,
        torch_dtype=torch.float32,
        quantization_config=bnb_config
    )

    return model

model = load_model()

Initializing BitsAndBytesConfig
Initializing model anakin87/zephyr-7b-alpha-sharded


Loading checkpoint shards: 100%|██████████| 8/8 [00:03<00:00,  2.37it/s]


In [3]:
def load_tokenizer():

    print('Initializing tokenizer')

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

tokenizer = load_tokenizer()

Initializing tokenizer


In [4]:
stop_token_ids = [0]

In [10]:
def initialize_chroma_db():

    print('Reading PDFs')

    # Read all files of the root folder
    loader = PyPDFDirectoryLoader('data')
    documents = loader.load()

    # Split documents in chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    all_splits = text_splitter.split_documents(documents)
    print(documents)
    # specify embedding model (using huggingface sentence transformer)
    embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
    model_kwargs = {"device": "cpu"}
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)

    # Create vector database and save the documents
    vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="./chroma_db")

    # specify the retriever
    retriever = vectordb.as_retriever()
    
    return retriever

retriever = initialize_chroma_db()

Reading PDFs
[Document(page_content='Golden Retriever \n \nArƟcle \nTalk \nRead \nView source \nView history \n \nTools \nThis is a good ar Ɵcle. Click here for more informa Ɵon. \nPage semi-protected \nFrom Wikipedia, the free encyclopedia \nFor other uses, see Golden Retriever (disambigua Ɵon). \nGolden Retriever \n \nOther names  \nFlat-coated Retriever, Golden \nYellow or Golden Retriever \nOrigin Scotland \nFoundaƟon stock  \nFlat-coated Retriever \nTweed Water Spaniel \nRed SeƩer \nBloodhound \nLabrador Retriever \nTraits \nHeight Males 56–61 cm (22–24 in) \nFemales 51–56 cm (20–22 in) \nWeight 25–34 kg (55–75 lb) ', metadata={'source': 'data\\golden.pdf', 'page': 0}), Document(page_content='Coat Flat or wavy double coat with good feathering, dense water-resistant undercoat \nColour Any shade of gold or cream \nLiƩer size 7.2±2.7[1]:\u200a4\u200a \nLife span 12–13 years \nKennel club standards \nThe Kennel Club standard \nFédéraƟon Cynologique Interna Ɵonale standard \nDog (domes

In [12]:
def build_pipeline():

    print('Initializing pipeline')

    model_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2048,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

    # specify the llm
    return HuggingFacePipeline(pipeline=model_pipeline)

pipeline = build_pipeline()

Initializing pipeline


In [15]:
def create_conversation(query: str, chat_history: list) -> tuple:
        
    try:

        memory = ConversationBufferMemory(memory_key='chat_history', return_messages=False)
        print(pipeline)
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=pipeline,
            retriever=retriever,
            memory=memory,
            get_chat_history=lambda h: h,
        )

        result = qa_chain({'question': query, 'chat_history': chat_history})
        chat_history.append((query, result['answer']))
        return '', chat_history


    except Exception as e:
        chat_history.append((query, e))
        return '', chat_history


In [16]:
resp, hius = create_conversation('What is a golden', [])


HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': None, 'pipeline_kwargs': None}


d:\Git\AIML_Capstone_Project\.venv\Lib\site-packages\transformers\generation\utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [18]:
resp, hius

('',
 [('What is a golden',
   ' A "golden" is a common name for the Golden Retriever dog breed. It is not a technical term\nin dog breeding or genetics, but simply a colloquial name based on the breed\'s characteristic golden coat.')])

In [2]:
import torch

print(torch.version)

<module 'torch.version' from 'd:\\Git\\AIML_Capstone_Project\\.venv\\Lib\\site-packages\\torch\\version.py'>
